In [9]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

# ==============================================================================
# [설정] 경로 및 파라미터 (변경됨)
# ==============================================================================
RAW_DATA_PATH_4H = "eth_4hour_all.csv"
RAW_DATA_PATH_5M = "eth_5min_krw.csv"
OUTPUT_LABEL_PATH = "eth_labels_3.0_1.5.csv"  # 결과 저장 파일명


ATR_MULTIPLIER_PROFIT = 3.0
ATR_MULTIPLIER_STOP = 1.5
FEE_RATE = 0.001 

# ==============================================================================
# [함수] 데이터 로드 (기존과 동일)
# ==============================================================================
def load_and_standardize(path):
    print(f"Loading {path}...")
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip().str.lower()
    
    time_cols = ['timestamp', 'date', 'time', 'datetime', 'open_time', 'opentime']
    for col in time_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])
            df.set_index(col, inplace=True)
            break
            
    rename_map = {}
    for col in df.columns:
        if 'eth_open' in col: rename_map[col] = 'open'
        elif 'eth_high' in col: rename_map[col] = 'high'
        elif 'eth_low' in col: rename_map[col] = 'low'
        elif 'eth_close' in col: rename_map[col] = 'close'
        elif 'eth_volume' in col: rename_map[col] = 'volume'
        
    if rename_map:
        df.rename(columns=rename_map, inplace=True)
    
    return df

# ==============================================================================
# [함수] PnL Engine (기존과 동일)
# ==============================================================================
def check_barrier_result(entry_time, entry_price, volatility, df_5m):
    # 4시간 후까지만 조회
    start_time = entry_time
    end_time = start_time + pd.Timedelta(hours=4)
    
    tp_price = entry_price + (volatility * ATR_MULTIPLIER_PROFIT)
    sl_price = entry_price - (volatility * ATR_MULTIPLIER_STOP)
    
    try:
        # 5분봉 슬라이싱 (속도 최적화: 인덱스 검색 최소화)
        # df_5m이 정렬되어 있다고 가정
        # loc은 느릴 수 있으므로, searchsorted를 쓰거나 그냥 loc을 쓰되 예외처리
        subset = df_5m.loc[start_time:end_time]
        
        if subset.empty: return 0, -FEE_RATE
        
        for t, row in subset.iterrows():
            # 안전하게 컬럼 접근
            h = row.get('high', row.get('eth_high'))
            l = row.get('low', row.get('eth_low'))
            c = row.get('close', row.get('eth_close'))
            
            # 1. 손절 체크
            if l <= sl_price:
                loss_pct = (sl_price - entry_price) / entry_price - FEE_RATE
                return 0, loss_pct # 실패
            
            # 2. 익절 체크
            if h >= tp_price:
                profit_pct = (tp_price - entry_price) / entry_price - FEE_RATE
                return 1, profit_pct # 성공
        
        # 3. 시간 초과 (Time Exit)
        last_row = subset.iloc[-1]
        exit_price = last_row.get('close', last_row.get('eth_close'))
        final_ret = (exit_price - entry_price) / entry_price - FEE_RATE
        

        return (1, final_ret) if final_ret > 0 else (0, final_ret)
        
    except Exception as e:
        return 0, -FEE_RATE

# ==============================================================================
# [Main] 정답지 생성 실행
# ==============================================================================
def generate_label_matrix():
    # 1. 데이터 로드
    df_4h = load_and_standardize(RAW_DATA_PATH_4H)
    df_5m = load_and_standardize(RAW_DATA_PATH_5M)
    

    start_date_5m = df_5m.index.min()
    print(f"\n[기간 조정] 5분봉 데이터 시작일: {start_date_5m}")
    
    df_4h = df_4h[df_4h.index >= start_date_5m]
    print(f" -> 4시간 봉 데이터 필터링 완료 (남은 개수: {len(df_4h)}개)")

    print("\n--- 데이터 준비 완료 ---")
    print(f"4H Data: {len(df_4h)} rows")
    print(f"5M Data: {len(df_5m)} rows")
    
    df_4h['tr'] = df_4h['high'] - df_4h['low']
    df_4h['ATR_Label'] = df_4h['tr'].rolling(14).mean().shift(1)  
    
    results = []
    
    print("\n--- 정답지 생성 시작 (전체 구간) ---")
    # tqdm으로 진행 상황 표시
    for i in tqdm(range(len(df_4h)), desc="Calculating Labels"):
        timestamp = df_4h.index[i]
        
        if i + 1 >= len(df_4h): continue 
        
        next_idx = i + 1
        next_timestamp = df_4h.index[next_idx]
        
  
        entry_price = df_4h.iloc[next_idx]['open']
        
        volatility = df_4h['tr'].rolling(14).mean().iloc[i]
        
        if pd.isna(volatility) or volatility == 0: continue
        
        # 5분봉에서 결과 확인
        # 진입 시간: next_timestamp (i+1번째 봉의 시작 시간)
        label, ret = check_barrier_result(next_timestamp, entry_price, volatility, df_5m)
        
        results.append({
            'signal_time': timestamp,      # 신호 발생 시간 (이때의 지표로 판단)
            'entry_time': next_timestamp,  # 실제 진입 시간
            'entry_price': entry_price,
            'volatility': volatility,
            'target': label,
            'return': ret
        })
        
    # 4. 결과 저장
    df_labels = pd.DataFrame(results)
    df_labels.set_index('signal_time', inplace=True)
    
    print("\n--- 정답지 생성 완료 ---")
    print(df_labels.head())
    print(f"Target 분포:\n{df_labels['target'].value_counts()}")
    
    # CSV 저장
    df_labels.to_csv(OUTPUT_LABEL_PATH)
    print(f"파일 저장 완료: {OUTPUT_LABEL_PATH}")
    return df_labels

# 실행
if __name__ == "__main__":
    generate_label_matrix()


Loading eth_4hour_all.csv...
Loading eth_5min_krw.csv...

[기간 조정] 5분봉 데이터 시작일: 2019-01-01 00:00:00
 -> 4시간 봉 데이터 필터링 완료 (남은 개수: 15170개)

--- 데이터 준비 완료 ---
4H Data: 15170 rows
5M Data: 725488 rows

--- 정답지 생성 시작 (전체 구간) ---


Calculating Labels: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15170/15170 [00:42<00:00, 359.30it/s]



--- 정답지 생성 완료 ---
                             entry_time  entry_price   volatility  target  \
signal_time                                                                 
2019-01-03 05:00:00 2019-01-03 09:00:00     171200.0  4828.571429       0   
2019-01-03 09:00:00 2019-01-03 13:00:00     171250.0  4575.000000       0   
2019-01-03 13:00:00 2019-01-03 17:00:00     171700.0  4617.857143       0   
2019-01-03 17:00:00 2019-01-03 21:00:00     166200.0  4989.285714       1   
2019-01-03 21:00:00 2019-01-04 01:00:00     167450.0  4950.000000       0   

                       return  
signal_time                    
2019-01-03 05:00:00 -0.000124  
2019-01-03 09:00:00 -0.000124  
2019-01-03 13:00:00 -0.041342  
2019-01-03 17:00:00  0.006822  
2019-01-03 21:00:00 -0.012944  
Target 분포:
0    8176
1    6980
Name: target, dtype: int64
파일 저장 완료: eth_labels_3.0_1.5.csv


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# ==============================================================================
# [설정] All-in-One 파라미터 조합 (Grid)
# ==============================================================================
# 1. 시간 (Time Horizons)
TIME_HORIZONS = [4, 12, 24, 48]  

RATIOS = [
    (1.0, 0.5, 'Scalp'),
    (2.0, 1.0, 'Classic'),
    (3.0, 1.5, 'Trend'),    
    (5.0, 2.0, 'Jackpot'),  
    (0.8, 0.8, 'Balanced')   
]

FEE_RATE = 0.0015
MIN_VOL_PCT = 0.005

# ==============================================================================
# [함수] 메인 생성기 (복잡도 O(N)으로 최적화)
# ==============================================================================
def generate_master_label_matrix():
    # 1. 데이터 로드 (기존 함수 활용)
    df_4h = load_and_standardize(RAW_DATA_PATH_4H)
    df_5m = load_and_standardize(RAW_DATA_PATH_5M)
    
    # 기간 동기화
    start_5m = df_5m.index.min()
    df_4h = df_4h[df_4h.index >= start_5m]
    
    # ATR 계산 (기준 변동성)
    df_4h['tr'] = df_4h['high'] - df_4h['low']
    df_4h['ATR_Ref'] = df_4h['tr'].rolling(14).mean()
    
    results = []
    print(f"\n--- 총 {len(TIME_HORIZONS) * len(RATIOS)}개 시나리오 동시 생성 시작 ---")
    
    # tqdm 적용
    for i in tqdm(range(len(df_4h)), desc="Generating Labels"):
        if i + 1 >= len(df_4h): continue
        
        # 기본 정보
        timestamp = df_4h.index[i]
        next_ts = df_4h.index[i+1] # 진입 시간
        entry_price = df_4h.iloc[i+1]['open']
        volatility = df_4h.iloc[i]['ATR_Ref']
        
        if pd.isna(volatility) or volatility == 0: continue
        
        # 5분봉 로드 (가장 긴 24시간치 한 번만 로드 -> 효율 극대화)
        max_h = max(TIME_HORIZONS)
        subset_full = df_5m.loc[next_ts : next_ts + pd.Timedelta(hours=max_h)]
        
        if subset_full.empty: continue
        
        # 결과 저장용 딕셔너리
        row_data = {
            'signal_time': timestamp,
            'entry_time': next_ts,
            'entry_price': entry_price,
            'volatility': volatility
        }
        
        # -------------------------------------------------------
        # [핵심] 2중 루프: 시간 x 손익비 조합
        # -------------------------------------------------------
        for hours in TIME_HORIZONS:
            # 해당 시간만큼 슬라이싱 (subset_full 재활용)
            end_t = next_ts + pd.Timedelta(hours=hours)
            subset = subset_full.loc[:end_t] 
            
            if subset.empty: continue
            
            for profit_mult, stop_mult, ratio_name in RATIOS:
                # 컬럼명 생성 예: target_4h_Trend, return_12h_Scalp
                col_suffix = f"{hours}h_{ratio_name}" 
                
                tp = entry_price + (volatility * profit_mult)
                sl = entry_price - (volatility * stop_mult)
                
                # Triple Barrier 로직
                target = 0
                ret = -FEE_RATE
                
                # 벡터화 검색 (루프보다 빠름) - pandas 조건 검색
                # 손절 터진 시간 찾기
                sl_hit = subset[subset['low'] <= sl]
                # 익절 터진 시간 찾기
                tp_hit = subset[subset['high'] >= tp]
                
                first_sl = sl_hit.index[0] if not sl_hit.empty else None
                first_tp = tp_hit.index[0] if not tp_hit.empty else None
                
                # 승패 판정
                if first_sl and first_tp:
                    # 둘 다 터졌으면 먼저 터진 게 우선
                    if first_sl < first_tp:
                        target = 0 # 손절
                        ret = (sl - entry_price) / entry_price - FEE_RATE
                    else:
                        target = 1 # 익절
                        ret = (tp - entry_price) / entry_price - FEE_RATE
                        
                elif first_sl: # 손절만 터짐
                    target = 0
                    ret = (sl - entry_price) / entry_price - FEE_RATE
                    
                elif first_tp: # 익절만 터짐
                    target = 1
                    ret = (tp - entry_price) / entry_price - FEE_RATE
                    
                else: # 둘 다 안 터짐 (Time Exit)
                    exit_price = subset.iloc[-1]['close']
                    ret = (exit_price - entry_price) / entry_price - FEE_RATE
                    target = 1 if ret > 0 else 0
                
                # 결과 저장
                row_data[f'target_{col_suffix}'] = target
                row_data[f'return_{col_suffix}'] = ret
                
        results.append(row_data)
        
    # CSV 저장
    df_out = pd.DataFrame(results)
    df_out.set_index('signal_time', inplace=True)
    
    filename = "eth_master_labels.csv"
    df_out.to_csv(filename)
    print(f"\n✅ 마스터 정답지 생성 완료: {filename}")
    print(f"총 데이터 수: {len(df_out)}")
    print("생성된 컬럼 예시:", [c for c in df_out.columns if 'target' in c][:5])

# 실행
if __name__ == "__main__":
    generate_master_label_matrix()


Loading eth_4hour_all.csv...
Loading eth_5min_krw.csv...

--- 총 9개 시나리오 동시 생성 시작 ---


Generating Labels:  95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍        | 14434/15170 [01:06<00:03, 216.53it/s]

In [13]:
TIME_HORIZONS = [4, 12, 24, 48]  

RATIOS = [
    (1.0, 0.5, 'Scalp'),
    (2.0, 1.0, 'Classic'),
    (3.0, 1.5, 'Trend'),
    (5.0, 2.0, 'Jackpot'),
    (0.8, 0.8, 'Balanced')
]

FEE_RATE = 0.0015
MIN_VOL_PCT = 0.005

def load_and_standardize(path):
    try:
        df = pd.read_csv(path)
    except FileNotFoundError:
        print(f"File not found: {path}")
        return pd.DataFrame()

    df.columns = df.columns.str.strip().str.lower()
    
    rename_map = {}
    for col in df.columns:
        if 'eth_' in col: rename_map[col] = col.replace('eth_', '')
    
    if rename_map:
        df.rename(columns=rename_map, inplace=True)
        
    time_col = [c for c in df.columns if 'time' in c or 'date' in c][0]
    df[time_col] = pd.to_datetime(df[time_col])
    df.set_index(time_col, inplace=True)
    df.sort_index(inplace=True)
    
    return df

def generate_master_label_matrix():
    df_4h = load_and_standardize(RAW_DATA_PATH_4H)
    df_5m = load_and_standardize(RAW_DATA_PATH_5M)
    
    if df_5m.empty:
        print("Error: 5-min data needed for precise labeling.")
        return

    start_common = max(df_4h.index.min(), df_5m.index.min())
    df_4h = df_4h[df_4h.index >= start_common]
    df_5m = df_5m[df_5m.index >= start_common]
    
    if 'high' in df_4h.columns and 'low' in df_4h.columns:
        df_4h['tr'] = df_4h['high'] - df_4h['low']
    else:
        df_4h['tr'] = df_4h['close'].diff().abs()
        
    df_4h['ATR_Ref'] = df_4h['tr'].rolling(14).mean()
    
    results = []
    print(f"Processing {len(TIME_HORIZONS) * len(RATIOS)} strategy scenarios...")
    
    for i in tqdm(range(len(df_4h))):
        if i + 1 >= len(df_4h): continue
        
        timestamp = df_4h.index[i]
        next_ts = df_4h.index[i+1]
        
        if next_ts not in df_4h.index: continue
        
        entry_price = df_4h.loc[next_ts, 'open']
        raw_vol = df_4h.iloc[i]['ATR_Ref']
        
        if pd.isna(raw_vol) or raw_vol == 0: continue
            
        min_vol = entry_price * MIN_VOL_PCT
        volatility = max(raw_vol, min_vol)
        
        max_h = max(TIME_HORIZONS)
        end_window = next_ts + pd.Timedelta(hours=max_h)
        subset_full = df_5m.loc[next_ts : end_window]
        
        if subset_full.empty: continue
        
        row_data = {
            'signal_time': timestamp,
            'entry_time': next_ts,
            'entry_price': entry_price,
            'volatility': volatility
        }
        
        for hours in TIME_HORIZONS:
            horizon_end = next_ts + pd.Timedelta(hours=hours)
            subset = subset_full.loc[:horizon_end]
            
            if subset.empty: continue
            
            for profit_mult, stop_mult, ratio_name in RATIOS:
                col_suffix = f"{hours}h_{ratio_name}"
                
                tp_price = entry_price + (volatility * profit_mult)
                sl_price = entry_price - (volatility * stop_mult)
                
                target = 0
                ret = 0.0
                
                sl_hits = subset.index[subset['low'] <= sl_price]
                tp_hits = subset.index[subset['high'] >= tp_price]
                
                first_sl = sl_hits[0] if len(sl_hits) > 0 else None
                first_tp = tp_hits[0] if len(tp_hits) > 0 else None
                
                if first_sl and first_tp:
                    if first_sl < first_tp:
                        ret = (sl_price - entry_price) / entry_price - FEE_RATE
                        target = 0
                    else:
                        ret = (tp_price - entry_price) / entry_price - FEE_RATE
                        target = 1
                elif first_sl:
                    ret = (sl_price - entry_price) / entry_price - FEE_RATE
                    target = 0
                elif first_tp:
                    ret = (tp_price - entry_price) / entry_price - FEE_RATE
                    target = 1
                else:
                    exit_price = subset.iloc[-1]['close']
                    ret = (exit_price - entry_price) / entry_price - FEE_RATE
                    target = 1 if ret > 0 else 0
                
                row_data[f'target_{col_suffix}'] = int(target)
                row_data[f'return_{col_suffix}'] = ret
        
        results.append(row_data)
        
    if not results:
        print("No label data generated.")
        return

    df_out = pd.DataFrame(results)
    df_out.set_index('signal_time', inplace=True)
    
    filename = "eth_master_labels_v2.csv"
    df_out.to_csv(filename)
    print(f"Saved: {filename} ({len(df_out)} rows)")

if __name__ == "__main__":
    generate_master_label_matrix()

Processing 20 strategy scenarios...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15170/15170 [01:25<00:00, 177.57it/s]


Saved: eth_master_labels_v2.csv (15156 rows)


In [19]:
import pandas as pd
import numpy as np

def verify_master_labels(csv_path, raw_5m_path):
    print(f"🔍 검증 시작: {csv_path}")
    
    # 1. 데이터 로드
    df_labels = pd.read_csv(csv_path)
    # 컬럼에 날짜가 문자열로 저장됐을 수 있으니 변환
    df_labels['signal_time'] = pd.to_datetime(df_labels['signal_time'])
    df_labels['entry_time'] = pd.to_datetime(df_labels['entry_time'])
    
    # 5분봉 로드 (검증용)
    print("Loading 5m data for audit...")
    df_5m = pd.read_csv(raw_5m_path)
    # 5분봉 컬럼 표준화 (간단 버전)
    df_5m.columns = df_5m.columns.str.strip().str.lower()
    time_cols = [c for c in df_5m.columns if 'time' in c or 'date' in c]
    if time_cols:
        df_5m[time_cols[0]] = pd.to_datetime(df_5m[time_cols[0]])
        df_5m.set_index(time_cols[0], inplace=True)
        
    rename_map = {}
    for c in df_5m.columns:
        if 'open' in c: rename_map[c] = 'open'
        elif 'high' in c: rename_map[c] = 'high'
        elif 'low' in c: rename_map[c] = 'low'
        elif 'close' in c: rename_map[c] = 'close'
    df_5m.rename(columns=rename_map, inplace=True)

    # =========================================================
    # [검사 1] 시간 인과성 체크
    # =========================================================
    print("\n[Test 1] Time Causality Check...")
    invalid_time = df_labels[df_labels['entry_time'] <= df_labels['signal_time']]
    if len(invalid_time) > 0:
        print(f"❌ 실패! 진입 시간이 신호 시간보다 빠르거나 같은 데이터 발견 ({len(invalid_time)}건)")
    else:
        print("✅ 통과! 모든 진입은 신호 발생 이후(미래)에 이루어짐.")

    # =========================================================
    # [검사 2] 타겟-수익률 논리 체크
    # =========================================================
    print("\n[Test 2] Logic Consistency Check (Target vs Return)...")
    # 모든 target 컬럼 찾기
    target_cols = [c for c in df_labels.columns if 'target_' in c]
    
    error_cnt = 0
    for t_col in target_cols:
        # 대응되는 return 컬럼 (예: target_4h_Scalp -> return_4h_Scalp)
        r_col = t_col.replace('target_', 'return_')
        
        # (1) Target=1 인데 Return < 0 인 경우 (수익이라며 왜 손실이야?)
        # 단, Time Exit에서 수수료 때문에 미세하게 마이너스인 경우는 제외해야 하나?
        # 로직상 ret > 0 이어야 target=1 이었음. (혹은 익절)
        # 단, check_barrier_result 로직에 따라 final_ret > 0 이면 1.
        
        # 여기선 엄격하게: Target=1 이면 Return > -FEE_RATE (수수료 감안해도 너무 큰 손실이면 에러)
        # 좀 더 간단하게: Target=1(성공)인데 수익률이 -1% 이하다? 이건 버그.
        inconsistent = df_labels[(df_labels[t_col] == 1) & (df_labels[r_col] < -0.005)] # 0.5% 이상 손실
        
        if len(inconsistent) > 0:
            print(f"⚠️ 경고: {t_col}=1 인데 큰 손실인 데이터 {len(inconsistent)}건 발견")
            print(inconsistent[[t_col, r_col]].head(1))
            error_cnt += 1
            
    if error_cnt == 0:
        print("✅ 통과! Target과 Return의 부호가 논리적으로 일치함.")

    # =========================================================
    # [검사 3] 무작위 샘플 실황 검증 (Spot Check)
    # =========================================================
    print("\n[Test 3] Random Spot Check (Deep Dive)...")
    # 검증할 시나리오 하나 픽 (예: 4h_Classic)
    test_scenario = '4h_Classic' # 4시간, 2.0:1.0
    target_col = f'target_{test_scenario}'
    
    if target_col not in df_labels.columns:
        test_scenario = target_cols[0].replace('target_', '') # 없으면 첫 번째 거
        target_col = target_cols[0]
        
    print(f"검증 대상 시나리오: {test_scenario}")
    
    # 무작위 3건 추출
    samples = df_labels.sample(3)
    
    for idx, row in samples.iterrows():
        entry_time = row['entry_time']
        entry_price = row['entry_price']
        vol = row['volatility']
        label = row[target_col]
        ret = row[f'return_{test_scenario}']
        
        # 시나리오 파싱 (예: 4h_Classic -> 4시간, 2.0, 1.0)
        # 이름에 규칙이 있다면 파싱, 아니면 수동 지정 (여기선 Classic=2.0:1.0 가정)
        # 복잡하니까 여기서는 'Classic'인 경우만 수동 검증한다고 가정
        profit_mult = 2.0
        stop_mult = 1.0
        hours = 4
        
        if 'Scalp' in test_scenario: profit_mult=1.0; stop_mult=0.5
        elif 'Trend' in test_scenario: profit_mult=3.0; stop_mult=1.5
        elif '12h' in test_scenario: hours=12
        elif '24h' in test_scenario: hours=24
        
        tp = entry_price + (vol * profit_mult)
        sl = entry_price - (vol * stop_mult)
        
        print(f"\n Case {idx}: {entry_time}")
        print(f"   Entry: {entry_price:.2f} | Vol: {vol:.2f}")
        print(f"   TP: {tp:.2f} | SL: {sl:.2f}")
        print(f"   Label: {label} | Return: {ret:.4%}")
        
        # 5분봉 확인
        end_time = entry_time + pd.Timedelta(hours=hours)
        subset = df_5m.loc[entry_time:end_time]
        
        if subset.empty:
            print("    5분봉 데이터 없음 (Skip)")
            continue
            
        high_max = subset['high'].max()
        low_min = subset['low'].min()
        
        print(f"   Actual High: {high_max:.2f} ({'Hit TP' if high_max >= tp else 'No'})")
        print(f"   Actual Low:  {low_min:.2f} ({'Hit SL' if low_min <= sl else 'No'})")
        
        
        if high_max >= tp and label == 0:
            # TP 쳤는데 실패? -> SL이 먼저 터졌어야 함
            if low_min <= sl:
                print("   정상 (TP 쳤지만 SL도 침 -> SL이 먼저 터진 것으로 추정)")
            else:
                print("    오류 가능성! (TP 쳤고 SL 안 쳤는데 왜 실패?)")
        elif low_min <= sl and label == 1:
            # SL 쳤는데 성공? -> TP가 먼저 터졌어야 함
            if high_max >= tp:
                print("    정상 (SL 쳤지만 TP도 침 -> TP가 먼저 터진 것으로 추정)")
            else:
                print("    오류 가능성! (SL 쳤고 TP 안 쳤는데 왜 성공?)")
        else:
            print("   ✅ 데이터 정합성 양호")

# 실행
verify_master_labels("eth_master_labels_v2.csv", "eth_5min_krw.csv")


🔍 검증 시작: eth_master_labels_v2.csv
Loading 5m data for audit...

[Test 1] Time Causality Check...
✅ 통과! 모든 진입은 신호 발생 이후(미래)에 이루어짐.

[Test 2] Logic Consistency Check (Target vs Return)...
✅ 통과! Target과 Return의 부호가 논리적으로 일치함.

[Test 3] Random Spot Check (Deep Dive)...
검증 대상 시나리오: 4h_Classic

📌 Case 14971: 2025-11-02 17:00:00
   Entry: 5799000.00 | Vol: 58285.71
   TP: 5915571.43 | SL: 5740714.29
   Label: 0 | Return: -0.4776%
   Actual High: 5809000.00 (No)
   Actual Low:  5745000.00 (No)
   ✅ 데이터 정합성 양호

📌 Case 12893: 2024-11-21 09:00:00
   Entry: 4338000.00 | Vol: 79857.14
   TP: 4497714.29 | SL: 4258142.86
   Label: 1 | Return: 0.7260%
   Actual High: 4383000.00 (No)
   Actual Low:  4309000.00 (No)
   ✅ 데이터 정합성 양호

📌 Case 1099: 2019-07-05 13:00:00
   Entry: 344050.00 | Vol: 7635.71
   TP: 359321.43 | SL: 336414.29
   Label: 1 | Return: 0.1843%
   Actual High: 347650.00 (No)
   Actual Low:  342850.00 (No)
   ✅ 데이터 정합성 양호


In [16]:
import pandas as pd
import numpy as np

RAW_DATA_PATH = "eth_4hour_all.csv"
LABEL_DATA_PATH = "eth_master_labels_v2.csv"

def verify_open_price_match():
    print(" [검증] '시가 진입(Next Open)' 정합성 테스트")
    
    # 1. 데이터 로드
    df_raw = pd.read_csv(RAW_DATA_PATH)
    df_raw.columns = df_raw.columns.str.strip().str.lower()
    rename_map = {c: c.replace('eth_', '') for c in df_raw.columns if 'eth_' in c}
    df_raw.rename(columns=rename_map, inplace=True)
    
    time_col = [c for c in df_raw.columns if 'time' in c or 'date' in c][0]
    df_raw[time_col] = pd.to_datetime(df_raw[time_col])
    df_raw.set_index(time_col, inplace=True)

    df_labels = pd.read_csv(LABEL_DATA_PATH)
    if 'entry_time' in df_labels.columns:
        df_labels['entry_time'] = pd.to_datetime(df_labels['entry_time'])
        df_labels.set_index('entry_time', inplace=True)
    else:
        l_time = [c for c in df_labels.columns if 'time' in c][0]
        df_labels[l_time] = pd.to_datetime(df_labels[l_time])
        df_labels.set_index(l_time, inplace=True)

    # 2. 검증: Raw Data의 'Open' vs Label의 'entry_price'
    # (Inner Join으로 시간 같은 것끼리 붙임)
    df_check = df_raw[['open']].join(df_labels[['entry_price']], how='inner')
    
    df_check['diff_pct'] = np.abs(df_check['open'] - df_check['entry_price']) / df_check['open']
    match_count = len(df_check[df_check['diff_pct'] < 0.001]) # 오차 0.1% 미만
    
    print(f"   총 {len(df_check)}건 중 {match_count}건 일치 (일치율: {match_count/len(df_check)*100:.1f}%)")
    
    if match_count / len(df_check) > 0.99:
        print(" [PASS] 완벽합니다. 'Entry Price'는 정확히 해당 시간의 'Open' 가격입니다.")
    else:
        print(" [FAIL] 불일치! 데이터가 밀렸거나 기준이 다릅니다. 확인 필요.")

if __name__ == "__main__":
    verify_open_price_match()


🕵️ [검증] '시가 진입(Next Open)' 정합성 테스트
   총 15156건 중 15156건 일치 (일치율: 100.0%)
✅ [PASS] 완벽합니다. 'Entry Price'는 정확히 해당 시간의 'Open' 가격입니다.


In [17]:
def verify_entry_logic():
    print(" [최종 검증] 진입 로직 무결성 테스트...")
    
    # 1. 원본 및 생성된 라벨 로드
    df_raw = load_and_standardize(RAW_DATA_PATH_4H)
    df_lbl = pd.read_csv("eth_master_labels_v2.csv")
    
    # 시간 컬럼 파싱
    df_lbl['signal_time'] = pd.to_datetime(df_lbl['signal_time'])
    df_lbl['entry_time'] = pd.to_datetime(df_lbl['entry_time'])
    
    error_count = 0
    total_checks = 0
    
    # 랜덤 샘플링 100개 검사 (전수조사는 시간이 걸리므로)
    sample_indices = np.random.choice(len(df_lbl), 100, replace=False)
    
    for idx in sample_indices:
        row = df_lbl.iloc[idx]
        sig_time = row['signal_time']
        ent_time = row['entry_time']
        ent_price = row['entry_price']
        
        # 1. 시간차 검증 (4시간 봉이면 정확히 4시간 차이여야 함)
        # 단, 주말/휴일 등으로 데이터가 비면 4시간 이상일 수 있음 -> Pass
        # 하지만 ent_time이 sig_time보다 '이전'이면 절대 안 됨.
        if ent_time <= sig_time:
            print(f"[FAIL] 시간 역행 발생! Signal: {sig_time}, Entry: {ent_time}")
            error_count += 1
            
        # 2. 가격 일치 검증
        # 원본 데이터에서 ent_time의 'Open' 가격 가져오기
        if ent_time in df_raw.index:
            raw_open = df_raw.loc[ent_time, 'open']
            
            # 부동소수점 오차 고려 (0.001% 이내)
            if abs(raw_open - ent_price) / raw_open > 0.00001:
                print(f" [FAIL] 가격 불일치! Time: {ent_time}, Raw Open: {raw_open}, Label Price: {ent_price}")
                error_count += 1
        else:
            print(f" [WARN] 원본에 없는 시간: {ent_time}")
            
        total_checks += 1
        
    if error_count == 0:
        print(f"\n [PASS] 검증 완료 ({total_checks}개 샘플). 무결성 확보됨.")
        print("   - 원칙 준수: Signal(t) -> Entry(t+1 Open)")
        print("   - 가격 정확성: 100% 일치")
    else:
        print(f"\n [FAIL] 총 {error_count}개의 오류가 발견되었습니다. 코드를 수정하세요.")

# 실행
if __name__ == "__main__":
    verify_entry_logic()


🕵️ [최종 검증] 진입 로직 무결성 테스트...

✅ [PASS] 검증 완료 (100개 샘플). 무결성 확보됨.
   - 원칙 준수: Signal(t) -> Entry(t+1 Open)
   - 가격 정확성: 100% 일치


In [22]:
dfff=pd.read_csv("./macro_data_4h/eth_4hour.csv")
dfff.tail(10)

timestamp     BTC_Open     BTC_High      BTC_Low  \
17926  2025-12-01 21:00:00  128939000.0  129742000.0  125809000.0   
17927  2025-12-02 01:00:00  126805000.0  128000000.0  125825000.0   
17928  2025-12-02 05:00:00  127299000.0  129998000.0  127299000.0   
17929  2025-12-02 09:00:00  129169000.0  130889000.0  128650000.0   
17930  2025-12-02 13:00:00  129858000.0  130375000.0  129758000.0   
17931  2025-12-02 17:00:00  130069000.0  131098000.0  129106000.0   
17932  2025-12-02 21:00:00  130746000.0  135958000.0  130123000.0   
17933  2025-12-03 01:00:00  135510000.0  137330000.0  134376000.0   
17934  2025-12-03 05:00:00  136806000.0  136900000.0  135000000.0   
17935  2025-12-03 09:00:00  135683000.0  136724000.0  135200000.0   

         BTC_Close   BTC_Volume   ETH_Open   ETH_High    ETH_Low  ETH_Close  \
17926  126792000.0  1211.100667  4245000.0  4245000.0  4085000.0  4118000.0   
17927  127295000.0   325.822107  4118000.0  4131000.0  4079000.0  4104000.0   
17928  129172000.0   384.533487  4107000.0  4223000.0  4105000.0  4193000.0   
17929  129826000.0   526.060156  4192000.0  4236000.0  4159000.0  4191000.0   
17930  130033000.0   193.514787  4190000.0  4204000.0  4176000.0  4194000.0   
17931  130738000.0   289.142313  4195000.0  4238000.0  4156000.0  4229000.0   
17932  135510000.0  1031.394144  4229000.0  4504000.0  4209000.0  4480000.0   
17933  136806000.0   557.165834  4480000.0  4518000.0  4446000.0  4486000.0   
17934  135626000.0   383.816387  4486000.0  4494000.0  4420000.0  4452000.0   
17935  136724000.0   216.333371  4453000.0  4484000.0  4436000.0  4484000.0   

       ...  fred_CPI_diff_1d  fred_CPI_pct_1d  fred_CPI_trend_20d  \
17926  ...             1.004         0.003105            0.022385   
17927  ...             1.004         0.003105            0.022385   
17928  ...             1.004         0.003105            0.022385   
17929  ...             1.004         0.003105            0.022385   
17930  ...             1.004         0.003105            0.022385   
17931  ...             1.004         0.003105            0.022385   
17932  ...             1.004         0.003105            0.022385   
17933  ...             1.004         0.003105            0.022385   
17934  ...             1.004         0.003105            0.022385   
17935  ...             1.004         0.003105            0.022385   

       fred_CPI_trend_60d  lido_eth_tvl  makerdao_eth_tvl  optimism_tvl  \
17926            0.089827  2.587541e+10      6.211846e+09   283995607.0   
17927            0.089827  2.587541e+10      6.211846e+09   283995607.0   
17928            0.089827  2.587541e+10      6.211846e+09   283995607.0   
17929            0.089827  2.578196e+10      6.108609e+09   289406101.0   
17930            0.089827  2.578196e+10      6.108609e+09   289406101.0   
17931            0.089827  2.578196e+10      6.108609e+09   289406101.0   
17932            0.089827  2.578196e+10      6.108609e+09   289406101.0   
17933            0.089827  2.578196e+10      6.108609e+09   289406101.0   
17934            0.089827  2.578196e+10      6.108609e+09   289406101.0   
17935            0.089827  2.417763e+10      6.139575e+09   278148982.0   

       uniswap_eth_tvl  usdt_eth_mcap  zksync_era_tvl  
17926     2.879735e+09   8.839346e+10      37583390.0  
17927     2.879735e+09   8.839346e+10      37583390.0  
17928     2.879735e+09   8.839346e+10      37583390.0  
17929     2.879735e+09   8.817305e+10      37018851.0  
17930     2.879735e+09   8.817305e+10      37018851.0  
17931     2.879735e+09   8.817305e+10      37018851.0  
17932     2.879735e+09   8.817305e+10      37018851.0  
17933     2.879735e+09   8.817305e+10      37018851.0  
17934     2.879735e+09   8.817305e+10      37018851.0  
17935     2.879735e+09   8.831738e+10      36157475.0  

[10 rows x 126 columns]

In [36]:
import pandas as pd

# 1. 파일 로드
df = pd.read_csv("eth_master_labels_v2.csv", index_col=0)

# 2. 각 전략별 1(Target)의 비율 계산
target_cols = [c for c in df.columns if 'target_' in c]
distribution = {}

for col in target_cols:
    counts = df[col].value_counts(normalize=True)
    win_rate = counts.get(1, 0) * 100  
    distribution[col] = win_rate

# 3. 결과 출력
print(f"{'Strategy Column':<30} | {'Win Rate (Label 1 %)'}")
print("-" * 55)
for col, rate in sorted(distribution.items(), key=lambda x: x[1]):
    print(f"{col:<30} | {rate:.2f}%")


Strategy Column                | Win Rate (Label 1 %)
-------------------------------------------------------
target_48h_Scalp               | 34.13%
target_24h_Scalp               | 34.82%
target_12h_Scalp               | 36.51%
target_48h_Classic             | 38.20%
target_4h_Scalp                | 39.65%
target_24h_Classic             | 41.76%
target_4h_Classic              | 43.16%
target_48h_Trend               | 43.51%
target_4h_Trend                | 43.70%
target_12h_Classic             | 43.75%
target_4h_Jackpot              | 43.79%
target_4h_Balanced             | 44.14%
target_12h_Trend               | 45.98%
target_24h_Trend               | 46.00%
target_48h_Jackpot             | 46.52%
target_12h_Jackpot             | 46.65%
target_24h_Jackpot             | 47.82%
target_12h_Balanced            | 48.32%
target_24h_Balanced            | 50.10%
target_48h_Balanced            | 50.77%


In [37]:
import pandas as pd
import os

RESULT_DIR = "backend_export_(14%버전_수정)"
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
FILE_PATH = os.path.join(RESULT_DIR, "backend_signals_master.csv")

def analyze_performance(file_path):
    if not os.path.exists(file_path):
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return

    print(">>> 데이터 로드 및 분석 중...")
    df = pd.read_csv(file_path)
    
    # 전략(Strategy) + 모델(Model)별 그룹화
    groups = df.groupby(['strategy', 'model'])
    
    results = []
    
    for (strat, model), sub in groups:
        # 1. 실제값과 예측값 추출
        y_true = sub['actual_direction']
        y_pred = sub['ml_prediction']
        
        # 2. 지표 계산 (zero_division=0: 분모가 0일 때 0 반환)
        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, zero_division=0)
        rec = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        
        # 3. 벤치마크(Raw) 승률 계산
        # (CUSUM이 찍어준 자리의 기본 승률)
        raw_win_rate = y_true.mean()
        
        # 4. 거래 횟수
        n_trades = y_pred.sum() # 모델이 '매수'라고 한 횟수
        total_opportunities = len(y_pred) # 전체 CUSUM 기회
        
        results.append({
            'Strategy': strat,
            'Model': model,
            'Accuracy': acc,
            'Precision (AI WinRate)': prec,
            'Raw WinRate (Benchmark)': raw_win_rate,
            'Delta (AI - Raw)': prec - raw_win_rate, # AI가 얼마나 개선했나?
            'Recall': rec,
            'F1 Score': f1,
            'Trades (AI Buy)': n_trades,
            'Total Events': total_opportunities,
            'Action Rate (%)': (n_trades / total_opportunities) * 100
        })
        
    # 데이터프레임 변환 및 정렬
    df_res = pd.DataFrame(results)
    
    # 보기 좋게 정렬 (Precision 높은 순)
    df_res = df_res.sort_values('Precision (AI WinRate)', ascending=False)
    
    # CSV로 저장
    #save_path = os.path.join(RESULT_DIR, "performance_analysis_report.csv")
    #df_res.to_csv(save_path, index=False)
    
    #print(f"✅ 분석 완료! 리포트 저장됨: {save_path}")
    print("\n[Top 5 Strategies by Precision]")
    print(df_res[['Strategy', 'Model', 'Precision (AI WinRate)', 'Raw WinRate (Benchmark)', 'Delta (AI - Raw)','Action Rate (%)']].head(10).to_string(index=False))
    return df_res
# 실행
df_res=analyze_performance(FILE_PATH)


>>> 데이터 로드 및 분석 중...

[Top 5 Strategies by Precision]
           Strategy Model  Precision (AI WinRate)  Raw WinRate (Benchmark)  Delta (AI - Raw)  Action Rate (%)
 target_48h_Jackpot    ET                1.000000                 0.475556          0.524444         0.222222
 target_24h_Jackpot    ET                0.692308                 0.471111          0.221197         1.444444
 target_12h_Jackpot    ET                0.687500                 0.482222          0.205278         1.777778
target_12h_Balanced   CAT                0.589041                 0.492222          0.096819        24.333333
target_48h_Balanced    ET                0.567010                 0.520000          0.047010        10.777778
target_12h_Balanced   XGB                0.544025                 0.492222          0.051803        35.333333
target_48h_Balanced   CAT                0.543478                 0.520000          0.023478        30.666667
target_12h_Balanced  LGBM                0.537267                 

In [40]:
df_res.head(10).sort_values("Precision (AI WinRate)",ascending=False)

Strategy Model  Accuracy  Precision (AI WinRate)  \
49   target_48h_Jackpot    ET  0.526667                1.000000   
29   target_24h_Jackpot    ET  0.534444                0.692308   
9    target_12h_Jackpot    ET  0.524444                0.687500   
0   target_12h_Balanced   CAT  0.551111                0.589041   
41  target_48h_Balanced    ET  0.494444                0.567010   
3   target_12h_Balanced   XGB  0.538889                0.544025   
40  target_48h_Balanced   CAT  0.506667                0.543478   
2   target_12h_Balanced  LGBM  0.534444                0.537267   
43  target_48h_Balanced   XGB  0.508889                0.531553   
21  target_24h_Balanced    ET  0.496667                0.529412   

    Raw WinRate (Benchmark)  Delta (AI - Raw)    Recall  F1 Score  \
49                 0.475556          0.524444  0.004673  0.009302   
29                 0.471111          0.221197  0.021226  0.041190   
9                  0.482222          0.205278  0.025346  0.048889   
0                  0.492222          0.096819  0.291196  0.389728   
41                 0.520000          0.047010  0.117521  0.194690   
3                  0.492222          0.051803  0.390519  0.454665   
40                 0.520000          0.023478  0.320513  0.403226   
2                  0.492222          0.045045  0.390519  0.452288   
43                 0.520000          0.011553  0.467949  0.497727   
21                 0.510000          0.019412  0.117647  0.192513   

    Trades (AI Buy)  Total Events  Action Rate (%)  
49                2           900         0.222222  
29               13           900         1.444444  
9                16           900         1.777778  
0               219           900        24.333333  
41               97           900        10.777778  
3               318           900        35.333333  
40              276           900        30.666667  
2               322           900        35.777778  
43              412           900        45.777778  
21              102           900        11.333333